### Four Types of Analytics
![chart](../images/4-types-of-data-analytics-01.png)

In [ ]:
#from shapely.geometry import Point
import pandas as pd
import numpy as np
import geopandas as gpd
#geopandas reads files with the `read_file()` method
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
%matplotlib inline
!pip install seaborn
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

#### Let's get some data and take a look

In [ ]:
NSSCrisis1_df = pd.read_csv('../Data/NSSCrisisv.1.csv')

In [ ]:
NSSCrisis2_df = pd.read_csv('../Data/NSSCrisisv.2.csv')

In [ ]:
CallCenter_2020 = pd.read_csv('../data/Contact_center_call_volume_2020.csv')

In [ ]:
CallCenter_2021 = pd.read_csv('../data/Contact_center_call_volume_2021.csv')

In [ ]:
CallCenter_2022 = pd.read_csv('../data/Contact center call volume_2022.csv')

In [ ]:
NSSCrisis1_df.shape

In [ ]:
NSSCrisis2_df.shape

In [ ]:
CallCenter_2020.shape

In [ ]:
CallCenter_2021.shape

In [ ]:
CallCenter_2022.shape

In [ ]:
NSSCrisis1_df.head()

In [ ]:
header_row = 1

In [ ]:
NSSCrisis1_df.columns = NSSCrisis1_df.iloc[header_row]

In [ ]:
NSSCrisis1_df = NSSCrisis1_df.drop(header_row)

In [ ]:
NSSCrisis1_df = NSSCrisis1_df.reset_index(drop=True)
NSSCrisis1_df.head()

In [ ]:
NSSCrisis2_df.head()

In [ ]:
NSSCrisis2_df.columns = NSSCrisis2_df.iloc[header_row]

In [ ]:
NSSCrisis2_df = NSSCrisis2_df.drop(header_row)

In [ ]:
NSSCrisis2_df = NSSCrisis2_df.reset_index(drop=True)
NSSCrisis2_df.head()

In [ ]:
NSSCrisis_df = pd.concat([NSSCrisis1_df, NSSCrisis2_df], axis=0, ignore_index=True)

In [ ]:
NSSCrisis_df.shape

In [ ]:
NSSCrisis_df.head()

In [ ]:
CallCenter_df_temp = pd.concat([CallCenter_2020, CallCenter_2021], axis=0, ignore_index=True)

In [ ]:
CallCenter_df_temp.shape

In [ ]:
CallCenter_df = pd.concat([CallCenter_df_temp, CallCenter_2022], axis=0, ignore_index=True)

In [ ]:
CallCenter_df.shape

In [ ]:
NSSCrisis_df.head()

In [ ]:
#header_row = 1

In [ ]:
#NSSCrisis_df.columns = NSSCrisis_df.iloc[header_row]

In [ ]:
#NSSCrisis_df = NSSCrisis_df.drop(header_row)

In [ ]:
#NSSCrisis_df = NSSCrisis_df.reset_index(drop=True)
#NSSCrisis_df.head()

In [ ]:
NSSCrisis_df = NSSCrisis_df.drop(0)

In [ ]:
NSSCrisis_df = NSSCrisis_df.reset_index(drop=True)

In [ ]:
NSSCrisis_df.head()

In [ ]:
print(list(NSSCrisis_df.columns))

In [ ]:
#index shows column index -- can be used to exclude columns with nan data
idx_dic = {}
for col in NSSCrisis_df.columns:
    idx_dic[col] = 63528 - NSSCrisis_df[col].isna().sum()
#print(idx_dic)
index_col_nan = pd.DataFrame(list(idx_dic.items()),columns = ['col_name','num_good_data'])
index_col_nan['%'] = ((index_col_nan['num_good_data'])/63528)*100
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
index_col_nan
#pd.reset_option('all')

In [ ]:
good_data_df = index_col_nan.sort_values(['%'], ascending=[False])
good_data_df

In [ ]:
NSSCrisis_df['PostalCode'].value_counts()
#df2=df[df['Fee']==22000]['Courses'].values[0]

In [ ]:
NSSCrisis_df['CRISIS Demographics - Military'].value_counts(normalize=True)

In [ ]:
NSSCrisis_df['CRISIS Demographics - Gender Identity'].value_counts()

### 1. Examine call volume and identify surge times, seasons, or events.

In [ ]:
#Read first row
NSSCrisis_df.iloc[0]

In [ ]:
NSSCrisis_df['Year'] = NSSCrisis_df['CallDateAndTimeStart'].str.extract('\d{2}/\d{2}/(\d{4})', expand=True)
NSSCrisis_df['Year'].head()

In [ ]:
NSSCrisis_df['Date'] = NSSCrisis_df['CallDateAndTimeStart'].str.extract('(\d{2}/\d{2}/\d{4})', expand=True)
NSSCrisis_df['Date'].head()

In [ ]:
NSSCrisis_df['Day'] = NSSCrisis_df['CallDateAndTimeStart'].str.extract('\d{2}/(\d{2})/\d{4}', expand=True)
NSSCrisis_df['Day'].head()

In [ ]:
NSSCrisis_df['Month'] = NSSCrisis_df['CallDateAndTimeStart'].str.extract('(\d{2})/\d{2}/\d{4}', expand=True)
NSSCrisis_df['Month'].head()

### 3. Assess regional and demographic trends in call content and call volume.

In [ ]:
#Looking at Military Data
NSSCrisis_df.groupby(['CRISIS Demographics - Military','Year'])['CallReportNum'].count()

In [ ]:
NSSCrisis_df.PostalCode.head(10)

In [ ]:
NSSCrisis_df.PostalCode.dtypes

In [ ]:
NSSCrisis_df['PostalCode'] = pd.to_numeric(NSSCrisis_df['PostalCode'], errors='coerce')

In [ ]:
NSSCrisis_df = NSSCrisis_df.dropna(subset=['PostalCode'])

In [ ]:
NSSCrisis_df.PostalCode.head()

In [ ]:
NSSCrisis_df.PostalCode.dtype

In [ ]:
#save this method ... astype int did not work
NSSCrisis_df = NSSCrisis_df.astype({'PostalCode':'int'})

In [ ]:
NSSCrisis_df.PostalCode.dtype

In [ ]:
zip_calls = NSSCrisis_df['PostalCode'].value_counts().sort_values(ascending=False).rename_axis('zip_code').reset_index(name='num_calls')
zip_calls.head()

In [ ]:
#add zip-median income data
tn_zip_median_income = pd.read_csv('../Data/tn_median_income_zip_transpose.csv')
tn_zip_median_income.head()

In [ ]:
tn_zip_median_income.drop(tn_zip_median_income.iloc[:, 2:631], inplace = True, axis = 1)
tn_zip_median_income.head()

In [ ]:
zip_calls.columns

In [ ]:
tn_zip_median_income.columns

In [ ]:
#left join income to zip calls
zip_combined = zip_calls.merge(tn_zip_median_income, left_on='zip_code', right_on='Zip_Code', how='left')
zip_combined.drop(zip_combined.iloc[:, 2:3], inplace = True, axis = 1)
zip_combined.head()

In [ ]:
#remove commas from Medium_Income
zip_combined["Median_Income"] = [str(i).replace(",", "") for i in zip_combined["Median_Income"]]
zip_combined.head()

In [ ]:
zip_combined.dtypes

In [ ]:
#change Median_Income to int
zip_combined['Median_Income'] = pd.to_numeric(zip_combined['Median_Income'], errors = 'coerce')

In [ ]:
zip_combined.dtypes

In [ ]:
#create scatter plot of num_calls vs. median income for zip_combined
#percentage of songwriters not listed by decade
plt.scatter(zip_combined.Median_Income, zip_combined.num_calls, alpha=0.5, cmap="viridis")
#plt.barh(writer_listed['decade'], writer_listed['Percent_Listed_No'], color='black')
plt.ylabel('Call Volume', fontsize=10, color='black', rotation=90, labelpad=15)
plt.title('Call Volume vs. Median Income \n (by Zip Code)', fontsize=12, fontname="Times New Roman Bold", color='black', pad=20)
#ax.set_title('T', pad=20)
plt.tick_params(axis='x', colors='black')
plt.tick_params(axis='y', colors='black')
plt.xlabel('Call Zip Code Median Income \n (dollars)', fontsize=11, color='black', rotation=0, labelpad=15)
plt.axvline(x=67521, color='r', linestyle='-', label=' U.S. Census \n National Household \n Median Income')
plt.legend(loc = 'upper right')
plt.savefig('median_income_v_calls_zip.png', dpi=200, transparent=False, bbox_inches='tight')
plt.show()

In [ ]:
#reset axis range
plt.xlim(2, 8)
plt.ylim(-0.50,1.5)

#create scatter plot of num_calls vs. median income for zip_combined
#percentage of songwriters not listed by decade
plt.scatter(zip_combined.num_calls, zip_combined.Median_Income, alpha=0.1, cmap="viridis")
plt.xlim(0,30)
plt.ylim(0,120000)
#plt.barh(writer_listed['decade'], writer_listed['Percent_Listed_No'], color='black')
plt.xlabel('Call Volume', fontsize=11, color='black', rotation=0, labelpad=15)
plt.title('Call Volume vs. Median Income \n (by Zip Code)', fontsize=12, fontname="Times New Roman Bold", color='black', pad=20)
plt.tick_params(axis='x', colors='black')
plt.tick_params(axis='y', colors='black')
plt.ylabel('Call Zip Code Median Income \n (dollars)', fontsize=11, color='black', rotation=90, labelpad=15)
plt.axhline(y=67521, color='r', linestyle='-', label='U.S. Census National Household Median Income')
plt.legend(loc = 'upper right',)
#plt.savefig('percent_songwriter_not_listed_by_decade.png', transparent=False)
plt.show()

In [ ]:
zip_combined.head()

In [ ]:
#read in lat-long for zip codes
zip_lat_lng = pd.read_csv('../Data/US Zip Codes from 2013 Government Data.csv', dtype={"zip":"string"})
zip_lat_lng.head()

In [ ]:
zip_combined.dtypes

In [ ]:
zip_combined = zip_combined.astype({'zip_code':'str'})

In [ ]:
zip_combined.dtypes

In [ ]:
zip_lat_lng.dtypes

In [ ]:
#join lat-lng to zip_combined
zip_combined_spat = zip_combined.merge(zip_lat_lng, left_on='zip_code', right_on='zip', how='left')
zip_combined_spat.head()

In [ ]:
zip_combined_spat.drop(zip_combined_spat.iloc[:, 3:4], inplace = True, axis = 1)
zip_combined_spat.head()

In [ ]:
#Veterans grouped by Zipcode
NSSCrisis_df.groupby(['CRISIS Demographics - Military','Year', 'PostalCode'])['CallReportNum'].count()

In [ ]:
#Veterans grouped by Zipcode
NSSCrisis_df.groupby(['PostalCode'])['CallReportNum'].count()

In [ ]:
NSSCrisis_df[NSSCrisis_df['PostalCode'] == '38012'].groupby(['Year'])['CallReportNum'].count()

In [ ]:
#looking at percentage of calls by zipcode
zip_count = NSSCrisis_df['PostalCode'].value_counts(normalize=False).sort_values(ascending=False)
zip_count

In [ ]:
#Value counts after group by example
#NSSCrisis_df.groupby('CRISIS Demographics - Education Level')['CRISIS Demographics - Occupation'].value_counts()


In [ ]:
#Looking at content of military calls
NSSCrisis_df[NSSCrisis_df['CRISIS Demographics - Military'] == 'Veteran'].groupby('CRISIS Demographics - Military')['CRISIS Issues - Emotional State'].value_counts().sort_values(ascending=False)

In [ ]:
#looking at homeless data
NSSCrisis_df['CRISIS Demographics - Homeless?'].value_counts()

In [ ]:
#homeless military relationship
NSSCrisis_df[NSSCrisis_df['CRISIS Demographics - Homeless?'] == 'Yes'].groupby('CRISIS Demographics - Military')['CRISIS Demographics - Military'].value_counts().sort_values(ascending=False)

In [ ]:
#homeless content
NSSCrisis_df[(NSSCrisis_df['CRISIS Demographics - Homeless?'] == 'Yes') & (NSSCrisis_df['CRISIS Issues - Emotional State'].str.contains('Anxious'))].groupby('CRISIS Issues - Emotional State')['CRISIS Issues - Emotional State'].count()

In [ ]:
#homeless content count for Anxious
NSSCrisis_df[(NSSCrisis_df['CRISIS Demographics - Homeless?'] == 'Yes')]['CRISIS Issues - Emotional State'].str.contains('Anxious').count()

In [ ]:
#homeless content count for Depressed
NSSCrisis_df[(NSSCrisis_df['CRISIS Demographics - Homeless?'] == 'Yes')]['CRISIS Issues - Emotional State'].str.contains('Depressed').count()

In [ ]:
#homeless calls by month
NSSCrisis_df[NSSCrisis_df['CRISIS Demographics - Homeless?'] == 'Yes'].groupby(['Year','Month'])['CallReportNum'].count()

In [ ]:
#homeless calls by month
NSSCrisis_df[NSSCrisis_df['CRISIS Demographics - Homeless?'] == 'Yes'].groupby(['Date'])['CallReportNum'].count()

### heatmap creation

In [ ]:
#read in tennessee geojson


In [ ]:
statemap = 'tl_2010_47_zcta510.shp'
#'../Data/tn_median_income_zip_transpose.csv'

In [ ]:
zips = gpd.read_file(statemap)

In [ ]:
zips.sample(5)

In [ ]:
#string replace
#regions['neighbourhood'] = regions['FIELD_7'].str.replace(' \(.+\)', '').str.lower()

In [ ]:
zips.plot(figsize=(40,20))
plt.show()

In [ ]:
#merge zips on ZCTA5CE10 column to zip_combined_spat zip_code column
merged = zips.set_index('ZCTA5CE10').join(zip_combined_spat.set_index('zip_code'))

In [ ]:
merged = merged.reset_index()

In [ ]:
merged = merged.fillna(0)

In [ ]:
#merged.sample(5)

In [ ]:
#fig, ax = plt.subplots(1, figsize=(40, 20))
ax.axis('off')
ax.set_title('Heat Map of Call Volume by Zip Code', fontdict={'fontsize': '40', 'fontweight' : '3'})

In [ ]:
color = 'BuPu'
vmin, vmax = 0, 231
sm = plt.cm.ScalarMappable(cmap=color, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)
cbar.ax.tick_params(labelsize=40)

In [ ]:
fig, ax = plt.subplots(1, figsize=(40, 20))
merged.plot('num_calls', cmap=color, linewidth=0.8, ax=ax, edgecolor='0.8', figsize=(40,20), categorical=True, legend=True)

### Common descriptive statistics
 - measures of central tendency (mean, median, mode)
 - measures of variability (standard deviation, variance)
 - distribution metrics (quartiles, interquartile range, outliers)

In [ ]:
people_df['weight'].value_counts()

#normalize = True finds percentage of bin
#sort = False
#ascending = True
#bins = 5
#.sort_index()

In [ ]:
pd.DataFrame(people_df['sex'].value_counts(dropna = False))

In [ ]:
people_df.describe()

#percentiles = [.1, .2, .3, .4, .5, .6, .7, .8, .9]
#include = 'all'

 -  What are the mean, median and mode for `weight`?

In [ ]:
# mean is 169.95
# median is 166
# mode is 140 -- most frequent
people_df.weight.value_counts()#.sort_index()

 - What are the following values for `years_played_sports`?
    - Minimum  
    - Maximum  
    - 1st Quartile  
    - 2nd Quartile  
    - 3rd Quartile  
    - Interquartile Range (IQR) (Difference between the 1st and 3rd quartiles) 

In [ ]:
people_desc = people_df.describe()
people_desc

In [ ]:
IQR = people_desc.loc['75%', 'years_played_sports'] - people_desc.loc['25%', 'years_played_sports']
print(IQR)

In [ ]:
#see below for explanation
low_outliers_s = 1 - (1.5 * 9.0)
print(low_outliers_s)
high_outliers_s = 10 + (1.5 * 9.0)
print(high_outliers_s)

#good way to select a single column value
low_outliers = people_desc.loc['25%', 'years_played_sports'] - (1.5 * IQR)
print(low_outliers)
high_outliers = people_desc.loc['75%', 'years_played_sports'] + (1.5 * IQR)
print(high_outliers)

In [ ]:
low_outliers_2 = people_desc.loc['25%', 'years_played_sports']-(1.5 * (people_desc.loc['75%', 'years_played_sports']-people_desc.loc['25%', 'years_played_sports']))

In [ ]:
low_outliers_2

- Outliers can be mathematically determined. They are values that fall below (Q1 − 1.5 IQR) or above (Q3 + 1.5 IQR)
    - how many outliers are there for `years_played_sports`?

In [ ]:
people_df.loc[:, 'years_played_sports'].sort_values()

In [ ]:
#how many outliers would we calculate?
print(len(people_df.loc[(people_df.years_played_sports < -12.5) | (people_df.years_played_sports > 23.5)]))
sns.boxplot(people_df.years_played_sports);

The length of the upper whisker is the largest value that is no greater than the third quartile plus 1.5 times the interquartile range. And the length of the lower whisker is the smallest value that is no less than the first quartile minus 1.5 times the interquartile range.

### [Correlations](https://statistics.laerd.com/statistical-guides/pearson-correlation-coefficient-statistical-guide.php) help us understand if variables _may_ have an underlying relationship
- a perfect positive correlation is 1.0 ; positive correlation means variables are moving in the same direction (when one increases, the other increases)
- a perfect negative correlation is -1.0 ; negative correlation means variables are moving in the opposite direction (when one increases, the other decreases)
- interpreting correlation depends on the context and purpose!


In [ ]:
people_df.corr()

 - Which variables in our dataset are most highly correlated with each other?
 - Which variables might explain the variability in `years_played_sports`?

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))

corr = people_df.corr()
# create a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap=cmap, mask = mask, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5});
#seaborn diverging palette

### Normalization
 - Does the `years_played_sports` variable have the same meaning for all people in the dataset? Is a 70 yo with 5 years of sports the same as a 20 yo with 5 years of sports?
 - How would you normalize it?

In [ ]:
#normalization is making data comparative
people_df['pct_life_sports'] = people_df.years_played_sports/people_df.age
people_df

In [ ]:
people_df.corr()

- Correlation between a variable you are trying to explain (sometimes called the dependent variable or target) and a variable that might explain it (independent variable or explanatory variable) helps us understand the target better.  
- Correlation between two explanatory variables may cause us to overestimate their importance to explaining the variance in the target. Think about _why_ variables might be highly correlated.
 

#### Feature Engineering

##### These could be from an external dataset or from the provided data

- compare bmi to an optimal bmi
- categorize as `oldest`, `middle`, `youngest`, or `only` child

In [ ]:
men_bmi_age = pd.read_csv('../data/men_bmi_age.csv')
women_bmi_age = pd.read_csv('../data/women_bmi_age.csv')

In [ ]:
men_bmi_age.head()

In [ ]:
women_bmi_age.head()

In [ ]:
people_df_men = people_df[people_df['sex'] == 'M']
people_df_women = people_df[people_df['sex'] == 'F']
people_df_unk = people_df[~people_df['sex'].isin(['M', 'F'])]
#opposite of what you're trying to find without it
people_df_unk_2 = people_df[people_df['sex'].isnull()]

In [ ]:
people_df_men.head()

In [ ]:
people_df_women.head()

In [ ]:
people_df_unk

In [ ]:
people_df_unk_2

In [ ]:
people_df_men = pd.merge(people_df_men, men_bmi_age, how = 'left', on = 'age')
people_df_women = pd.merge(people_df_women, women_bmi_age, how = 'left', on = 'age')

In [ ]:
people_df_men.head()

In [ ]:
people_df_men['diff_optimal_bmi'] = people_df_men['optimal_bmi'] - people_df_men['bmi']
people_df_women['diff_optimal_bmi'] = people_df_women['optimal_bmi'] - people_df_women['bmi']

In [ ]:
people_df_men

In [ ]:
people_df = pd.concat([people_df_women, people_df_men, people_df_unk])

In [ ]:
people_df

In [ ]:
people_df = people_df.reset_index(drop = True)

In [ ]:
people_df['birth_category'] = ''

In [ ]:
people_df

In [ ]:
for ind, row in people_df.iterrows():
    print('working on row: ', ind)
    if row['sibling_count'] == 0:
        people_df.loc[ind, 'birth_category'] = 'only'
    elif row['birth_order'] == 1:
        people_df.loc[ind, 'birth_category'] ='oldest'
    elif row['birth_order'] > row['sibling_count']:
        people_df.loc[ind, 'birth_category'] ='youngest'
    else:
        people_df.loc[ind, 'birth_category'] ='middle'
print('finished!') #need to have this to see progress

In [ ]:
people_df